In [1]:
import os
import pandas as pd
import pymssql
from IPython.display import clear_output

In [2]:
transaction_code = pd.read_excel('D:/NCTU/10-資料範例與欄位說明/NCTU_AI_AML資料表欄位說明_v20210507.xlsx', sheet_name= '交易代碼表')
credit_code = transaction_code.loc[transaction_code['交易方向']== 'CREDIT', '交易代碼'].values.astype(int)
debit_code = transaction_code.loc[transaction_code['交易方向']== 'DEBIT', '交易代碼'].values.astype(int)

debit_code 3168 excel 裡面主要類型是空白，次要類型是"無"，且沒有其他備註

In [4]:
debit_code

array([3168, 3171, 3174, 3177, 3178, 3181, 3183, 3185, 3186, 3189, 3192,
       3193, 3195, 3198, 3201, 3204, 3205, 3207, 3208, 3210, 3212, 3213,
       3214, 3215, 3216, 3219, 3222, 3223, 3224, 3225, 3226, 3228, 3231,
       3234, 3237, 3240, 3243, 3246, 3285, 3288, 3291, 3294, 3297, 3300,
       3303, 3306, 3309, 3312, 3315, 3318, 3321, 3330, 3410, 3413, 3416,
       3419, 3422, 3425, 3600, 3610, 3620, 5410, 5411, 5415, 5420, 5425,
       5430, 5435, 5440, 5445, 5450, 5455, 5460, 5465, 5470, 5540, 5545,
       8001, 8002, 8004, 8005, 8010, 8011, 8014, 8016, 8021, 8028, 8029,
       8030, 8031, 8032, 8033, 8034, 8047, 8048, 8049, 8050, 8051, 8052,
       8053, 8054, 8055, 8056, 8057, 8058, 8070, 8071, 8072, 8073, 8074,
       8075, 8076, 8077, 8080, 8084, 8085, 8087, 8096, 8098, 8102, 8104,
       8106, 8107, 8113, 8115, 8117, 8121, 5412, 9006])

In [5]:
conn = pymssql.connect(server= '.', user= 'NCTUUser2', password= 'NCTU@MetaEdge', database= 'NCTU(1090401-1091231)')
cursor = conn.cursor()

In [6]:
def transactionKeySAR(scenario_name):
    cursor.execute(f'SELECT Txn_Type_Key FROM dbo.SAR_Info_TxnTypeKey WHERE Scenario_name = {scenario_name}')
    txnKey = []

    for row in cursor:
        txnKey.append(int(row[0]))
    return txnKey

In [ ]:
end_of_file = '_txn.csv'

In [7]:
FEATURE = ['Credit_Amt', 'number_of_Credit', 'Debit_Amt', 'number_of_Debit', 'debit/credit']
def aggregation(df, day, scenario_name, party, feature, path, save_path):
    print(scenario_name + 'txn aggregation... start!')
    data = []
    txnKey = transactionKeySAR(scenario_name)

    for i in range(df.shape[0]):
        data_slice = []

        if i % 1000 == 0:
            clear_output(wait= True)
    
        txn_file_path = path + day + '/' + str(i) + end_of_file
        print('row account:', i, 'file path:', txn_file_path)
        print('scenario name:', df['scenario_name'][i], 'corresponding txn key:', txnKey)

        if os.path.isfile(txn_file_path):
            txn_file = pd.read_csv(txn_file_path)
            if txn_file.empty is False:
                credit_idx = []
                debit_idx = []

                for j in range(txn_file.shape[0]):
                    if party == 'ACC':
                        if txn_file['Acct_No'][j] == df['Acct_No'][i] and txn_file['Txn_Sts_Key'][j] == 1:
                            if txn_file['Txn_Type_KEy'][j] in txnKey and txn_file['Txn_Type_Key'][j] in credit_code:
                                print('Txn type key:', txn_file['Txn_Type_KEy'][j])
                                credit_idx.append(j)
                            elif txn_file['Txn_Type_Key'][j] in txnKey and txn_file['Txn_Type_Key'][j] in debit_code:
                                print('txn type key:', txn_file['Txn_Type_Key'][j])
                                debit_idx.append(j)
                    
                    elif party == 'PTY':
                        if txn_file['Txn_Sts_Key'][j] == 1:
                            if txn_file['Txn_Type_Key'][j] in txnKey and txn_file['Txn_Type_Key'][j] in debit_code:
                                print('txn type key:', txn_file['Txn_Type_Key'][j])
                                credit_idx.append(j)
                            elif txn_file['Txn_Type_Key'][j] in txnKey and txn_file['Txn_Type_Key'][j] in debit_code:
                                print('txn type key:', txn_file['Txn_Type_Key'][j])
                                debit_idx.append(j)

                credit = txn_file.iloc[credit_idx]
                credit_amt = credit['Ccy_Amt'].sum()
                credit_num = credit.shape[0]

                debit = txn_file.iloc[debit_idx]
                debit_amt = debit['Ccy_Amt'].sunm()
                debit_num = debit.shape[0]

                data_slice = [credit_amt, credit_num, debit_amt, debit_num]

                if credit_amt != 0:
                    data_slice.append(debit_amt/ credit_amt)
                else:
                    data_slice.append(-1)
            
            else:
                print(f'{str(i)}_txn.csv is empty')
            print('slice:', data_slice)
            data.append(data_slice)

    df1 = pd.DataFrame(data= data, columns= FEATURE)
    usecol = [*list(df.columns)]
    df = df[usecol]
    df = df[df['scenario_name'] == scenario_name]
    df.to_csv(save_path + scenario_name + '_' + day + '_with_aggregated_txn_type+key_營業日.csv', index= False)
    print(scenario_name + 'txn aggregation... done!')
    

In [ ]:
FEATURE = ['credit_Amt', 'number_of_Credit', 'Debit_Amt', 'number_of_Debit', 'debit/credit']

def aggregationScenarioGroup(df, group, day, path, save_path):
    print('Group txn aggregation... start!')
    data = []
    